# Robust methods for Machine Learning

## Adversarial attacks

#### Tutorial #2 (Anne Gagneux)

In [ ]:
# imports
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
import json

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
from torchvision import datasets, transforms


from tqdm import trange
torch.manual_seed(0)

We are going to consider a neural network $h_\theta$ (for classification) that has been trained on  $\mathbf X_{\text{train}}$ with the corresponding labels $y_{\text{train}}$. The general idea for *adversarial attacks* is to create $\mathbf x^{adv}$ by adding a small pertubation that changes the prediction. 

What is the intuition behind adversarial attacks ? How to explain that classifiers have both such good performance and yet fail when one adds little perturbations?

One of the hypotheses can be summed up as follows:

> Clean data lies in a low-dimensional manifold. Even though the adversarial examples are close to the clean data, they lie off the underlying data manifold.

In other words, the neural network learns the underlying (hidden) manifold of real-life images. Even small perturbations get the sample out of this manifold that is totally unknown to the neural network (and unseen during training). 

![Image manifold](https://www.researchgate.net/publication/347339813/figure/fig1/AS:1020696654249985@1620364454164/We-illustrate-the-notion-of-a-hidden-manifold-in-input-space-using-CIFAR10-example.png)


First, we load an image that we are going to misclassify. We use ImageNet pretrained network.

In [ ]:
# read the image, resize to 224 and convert to PyTorch Tensor
img = Image.open("beagle.jpg")
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
beagle_image = preprocess(img)[None, :, :, :]

# plot image (note that numpy using HWC whereas Pytorch user CHW, so we need to convert)
plt.imshow(beagle_image[0].numpy().transpose(1, 2, 0))

Now, we load a classification model (Resnet50) pretrained on ImageNet (1000 classes).

In [ ]:
# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)

    def forward(self, x):
        return (x - self.mean.type_as(x)[None, :, None, None]) / self.std.type_as(x)[None, :, None, None]


# values are standard normalization for ImageNet images,
# from https://github.com/pytorch/examples/blob/master/imagenet/main.py
norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# load pre-trained ResNet50, and put into evaluation mode (necessary to e.g. turn off batchnorm)
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.eval()

Now, let's predict the class of our image.

In [ ]:
pred = model(norm(beagle_image))
with open("imagenet_class_index.json") as f:
    imagenet_classes = {int(i): x[1] for i, x in json.load(f).items()}
print(imagenet_classes[pred.argmax().item()])
beagle_class = pred.argmax().item()

Some useful functions

In [ ]:
def perform_attack(model, image, image_adv):
    """ For a given attack, create an adversarial image and compare the predictions of the model on the original image and the adversarial one"""

    pred = model(norm(image))
    max_class = pred.argmax().item()
    print("Predicted class of before attack: ", imagenet_classes[max_class])
    print("Predicted probability of true class before attack: ",
          nn.Softmax(dim=1)(pred)[0, max_class].item())

    pred_adv = model(norm(image_adv))
    max_class_adv = pred_adv.argmax().item()
    print("")
    print("Predicted class after attack: ", imagenet_classes[max_class_adv])
    print("Predicted probability of true class after attack", nn.Softmax(
        dim=1)(pred_adv)[0, max_class].item())
    print("Predicted probability of new class", nn.Softmax(
        dim=1)(pred_adv)[0, max_class_adv].item())

    print("")
    if max_class_adv != max_class:
        print("The attack was successful !")
    else:
        print("The attack did not succeed.")


def perform_target_attack(model, image, image_adv, y_target):

    pred = model(norm(image))
    max_class = pred.argmax().item()
    print("Predicted class of before attack: ", imagenet_classes[max_class])
    print("Predicted probability of true class before attack: ",
          nn.Softmax(dim=1)(pred)[0, max_class].item())
    print("Predicted probability of target class before attack: ",
          nn.Softmax(dim=1)(pred)[0, y_target].item())

    pred_adv = model(norm(image_adv))
    max_class_adv = pred_adv.argmax().item()

    print("")
    if max_class_adv == y_target:
        print("The attack was successful !")
        print("")
        print("Predicted class after attack: ",
              imagenet_classes[max_class_adv])
        print("Predicted probability of true class after attack", nn.Softmax(
            dim=1)(pred_adv)[0, max_class].item())
        print("Predicted probability of target class", nn.Softmax(
            dim=1)(pred_adv)[0, max_class_adv].item())
    else:
        print("The attack did not succeed.")
        print("")
        print("Predicted class after attack: ",
              imagenet_classes[max_class_adv])
        print("Predicted probability of true class after attack", nn.Softmax(
            dim=1)(pred_adv)[0, max_class].item())
        print("Predicted probability of target class", nn.Softmax(
            dim=1)(pred_adv)[0, y_target].item())
        print("Predicted probability of adv class", nn.Softmax(
            dim=1)(pred_adv)[0, max_class_adv].item())


def display_attack(image, image_adv):
    """Display the adversarial example for a given attack"""
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow((image)[0].detach().numpy().transpose(1, 2, 0))
    axs[1].imshow((image_adv)[0].detach().numpy().transpose(1, 2, 0))
    plt.show()


def display_diff(image, image_adv):
    fig = plt.figure(figsize=(6, 4))
    diff = (image_adv-image)[0].detach()
    plt.imshow((torch.abs(diff)/(torch.max(diff)-torch.min(diff))
                ).numpy().transpose(1, 2, 0))
    plt.xticks([])
    plt.yticks([])
    plt.title(r"$|x - x^{adv}|$")
    plt.show()

### "White-box" attacks
White-box attacks mean we have *access to the full network*. Our goal is to slightly modify the image, in such a way that a human does not notice it but that a neural network is no longer capable of correctly classifying the image.

##### Random pertubation
Random pertubation is the most trivial attack. One can randomly perturb the image pixel by pixel within a given range.
It writes as:
$$ \mathbf x^{adv} = \mathbf x + \mathcal U([-\epsilon, \epsilon])$$

In [ ]:
def random_perturbation(X, epsilon):
    """ Construct randomly pertubed adversarial examples on the examples X"""
    delta = # TO COMPLETE
    x_adv = X + delta
    return x_adv.clip(0, 1)

In [ ]:
# perform the attack
epsilon = # TO COMPLETE: epsilon in [0,1]. The greater the epsilon, the more visible the pertubation becomes. Try different values.
image_adv = random_perturbation(beagle_image, epsilon=epsilon)
perform_attack(model, beagle_image, image_adv)
display_attack(beagle_image, image_adv)
display_diff(beagle_image, image_adv)

#### Gradient based adversarial attacks

When training a neural network, ones minimize the loss $\ell(\mathbf X_{\text{train}}, y_{\text{train}})$ using back-propagation. \
These gradients are used to update the weights of the neural network. \
Now, we consider a trained neural network. Here, we want to maximize the loss (in order to force misclassification). \
Instead of updating the weights, we want to update the image itself, i.e. tune the pertubation $\delta$ in order to misclassify the image $\mathbf x$.

*Notations:*
In the following $h_\theta$ denotes the model with weights $\theta$.

##### Fast gradient sign method (FGSM)

The idea is to adjust the pertubation $\delta$ in the direction of the gradient in order to maximise the loss. \
One wish to do the larger step as possible, so to always move by $\pm \epsilon$.

It writes as:
$$ \mathbf x^{adv} = \mathbf x + \epsilon \cdot \mathrm{sign} \left( \nabla_{\mathbf x} \ell (\mathbf x, y) \right) $$


In [ ]:
def fgsm(model, X, y, epsilon):
    """ Construct randomly pertubed adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    criterion = nn.CrossEntropyLoss()
    pred = model(norm(X+delta))
    loss = # TO COMPLETE 
    loss.backward()
    x_adv = # TO COMPLETE
    return x_adv.clip(0, 1)

In [ ]:
# perform the attack
image_adv = fgsm(model, beagle_image, beagle_class, epsilon=10./255.)
perform_attack(model, beagle_image, image_adv)
display_attack(beagle_image, image_adv)
display_diff(beagle_image, image_adv)

#### Projected gradient descent (PGD)

We write tha attack as a constrained optimization problem:
$$ \max_{\delta \in \Delta} l(h_\theta(\mathbf x+\delta),y) $$

**PGD iterations:**
$$\delta_{t+1} = \mathcal P_\Delta ( \delta_t + \eta_t \nabla_\delta l(h_\theta(\mathbf x +\delta_t,y)) )$$

One needs to compute the projection onte the set $\Delta$.

- If $\Delta$ is the $\ell_\infty$ ball, i.e. $\Delta = \mathcal B_\infty (0,\epsilon) = \{ \mathbf x : \max_i {|x_i|} \leq \epsilon\}$, the projection simply writes as  clipping values to $[-\epsilon, \epsilon]$.

- If $\Delta$ is the $\ell_2$ ball, i.e. $\Delta = \mathcal B_2 (0,\epsilon) = \{ \mathbf x : \Vert x \Vert_2 \leq \epsilon\}$, the projection writes as:
$$ \mathcal P_{\ell_2} ( a )  = \frac{a}{ \max \left( \frac{\Vert a \Vert}{\epsilon} , 1 \right)} $$ 



In [ ]:
def projection_Linfty(x, epsilon):
    # TO COMPLETE
    return None


def projection_L2(x, epsilon):
    # TO COMPLETE
    return None


def pgd(model, X, y, epsilon, projection, n_steps=40, eta=5e-3):
    """ Run projected gradient descent on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    for k in trange(n_steps):
        criterion = nn.CrossEntropyLoss()
        pred = model(norm(X+delta))
        loss = # TO COMPLETE
        loss.backward()
        delta.data = # TO COMPLETE
        delta.grad.zero_()  # do not forget to put the gradient back to 0 before the next step
    x_adv = X + delta
    return x_adv.clip(0, 1)

In [ ]:
# perform the attack
image_adv = pgd(model, beagle_image, beagle_class,
                epsilon=0.5, projection=projection_L2)
perform_attack(model, beagle_image, image_adv)
display_attack(beagle_image, image_adv)
display_diff(beagle_image, image_adv)

In [ ]:
# perform the attack
image_adv = pgd(model, beagle_image, beagle_class,
                epsilon=10./255., projection=projection_Linfty, eta=0.1)
perform_attack(model, beagle_image, image_adv)
display_attack(beagle_image, image_adv)
display_diff(beagle_image, image_adv)

To go further, you can try with $\Delta$ the $\ell_1$ ball, i.e. $\Delta = \mathcal B_1 (0,\epsilon) = \{ \mathbf x : \sum_i |x_i|\leq \epsilon\}$. You first need to compute the projection operator (you can  write the Lagrangian and use KKT conditions).

In [ ]:
def projection_L1(x, epsilon):
    original_shape = x.shape
    x = x.view(x.shape[0], -1)
    mask = (torch.norm(x, p=1, dim=1) < epsilon).float().unsqueeze(1)
    mu, _ = torch.sort(torch.abs(x), dim=1, descending=True)
    cumsum = torch.cumsum(mu, dim=1)
    arange = torch.arange(1, x.shape[1] + 1, device=x.device)
    rho, _ = torch.max((mu * arange > (cumsum - epsilon)) * arange, dim=1)
    theta = (cumsum[torch.arange(x.shape[0]), rho.cpu() - 1] - epsilon) / rho
    proj = (torch.abs(x) - theta.unsqueeze(1)).clamp(min=0)
    x = mask * x + (1 - mask) * proj * torch.sign(x)
    return x.view(original_shape)

In [ ]:
# perform the attack
image_adv = pgd(model, beagle_image, beagle_class,
                epsilon=0.2, projection=projection_L1, eta=5e-3)
perform_attack(model, beagle_image, image_adv)
display_attack(beagle_image, image_adv)
display_diff(beagle_image, image_adv)

What do you observe ? Look at the average difference between the original image and the target image ? What can you say ? 

In [ ]:
(torch.abs(beagle_image-image_adv)).mean()

#### Targeted attack

What we have considered so far are “untargeted” attacks, meaning they effectively try to change the label to any alternative, rather than change it to a particular alternative. If we want to target the attack to a specific laabel, we can both maximize the loss on the true label and minimize the loss on the target label.

It writes:

$$ \max_{\delta \in \Delta} l(h_\theta(\mathbf x+\delta),y_{\text{truth}}) - l(h_\theta(\mathbf x+\delta),y_{\text{target}})$$ 


In [ ]:
def fgsm_target(model, X, y, y_target, epsilon):
    """ Construct randomly pertubed adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    criterion = nn.CrossEntropyLoss()
    pred = model(norm(X+delta))
    loss = # TO COMPLETE
    loss.backward()
    x_adv = # TO COMPLETE
    return x_adv

In [ ]:
def pgd_target(model, X, y, y_target, epsilon, projection, n_steps=100, eta=5e-3):
    """ Run projected gradient descent on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    
    for k in trange(n_steps):
        criterion = nn.CrossEntropyLoss()
        pred = model(norm(X+delta))
        loss = # TO COMPLETE
        loss.backward()
        delta.data = # TO COMPLETE
        delta.grad.zero_()  # do not forget to put the gradient back to 0 before the next step
    x_adv = X + delta
    return x_adv.clip(0, 1)

Choose a class of your choice looking in the imagenet_class_index.json.
Try to change the beagle_image for the net to predict with your chosen class. 
Try for different methods, different classes, different range of $\epsilon$.

In [ ]:
# perform the attack
image_adv = pgd_target(model, beagle_image, beagle_class, y_target=330,
                       epsilon=0.2, projection=projection_Linfty)
perform_target_attack(model, beagle_image, image_adv, 330)
display_attack(beagle_image, image_adv)
display_diff(beagle_image, image_adv)

## Black-box attack

Here, we do not have access to the weights of the model anymore. 
Thus, we can not perform back-propagation to find the best perturbation. \
In the black-box setting, an attacker can only access outputs of the target model. Based on whether one has access to the full probability or the label of a given input, black-box attacks are further divided into score-based and decision-based.

#### HopSkipJump Attack: a decision-based adversarial attack

*Reference: [HopSkipJumpAttack: A Query-Efficient Decision-Based Adversarial Attack](https://arxiv.org/abs/1904.02144) by Jianbo Chen, Michael I. Jordan, Martin J. Wainwright.*

We suppose we have access to the output labels of the model we want to attack, i.e. we only have access to the prediction of an input $\mathbf x$. It is the most restrictive setting.

We do not have access to the gradient: the HopSkipJump attacks proposes to estimate this gradient.

Let $m$ be the number of classes.
We want the net to misclassify our input $\mathbf x$.
So, if $c = \arg \max_{i \in [1,m]} h_\theta(\mathbf x)$ is the predicted class, we want that:
$$\arg \max_{i \in [1,m]} h_\theta(\mathbf x^{adv}) \neq c$$
which rewrites as:
$$S_{\mathbf x} (\mathbf x^{adv})  = \max_i  [ h_\theta(\mathbf x^{adv})]_i - [ h_\theta(\mathbf x^{adv})]_c > 0 $$


The goal of an adversarial attack is to generate a perturbed sample $\mathbf x^{adv}$ such that $S_{\mathbf x} (\mathbf x^{adv}) > 0$, while keeping $\mathbf x^{adv}$ close to the original sample $\mathbf x$. 
This can be formulated as the optimization problem:
$$ \min_{\mathbf x'} d(\mathbf x', \mathbf x) \text{ such that } S_{\mathbf x} (\mathbf x') > 0$$

For the $\ell_2$ distance:
$$d(\mathbf x', \mathbf x) = \Vert x- x' \Vert_2$$

We are given an image $\mathbf x^*$ we want to attack (i.e. to modify slightly for the model to make a wrong prediction).
The algorithm takes an other misclassified image $x_0$ such that $S_{\mathbf x} (x_0) > 0$ and updates its pixels in the following iterative way:

$$ \mathbf x_{t+1} = \alpha_t \mathbf x^* + (1 - \alpha_t) (\mathbf x_t + \eta \nabla S_{\mathbf x} (\mathbf x_t) ) $$

**BUT how to compute $ \nabla S_{\mathbf x} (\mathbf x_t)$ ?**

Monte-Carlo estimate at the boundary (when $S_{\mathbf x} (\mathbf x_t) \sim 0$):

$$\widetilde{\nabla S_{\mathbf x}} (\mathbf x_t) := \frac{1}{B} \sum_{b=1}^B \mathbf 1_{S_{\mathbf x} (\mathbf x_t + \delta u_b)>0} u_B$$
where $\{ u_b \}_{b=1}^B$ are i.i.d. draws from the uniform distribution over the unit sphere.  

In [ ]:
def initialize(model, norm, X, y, thresh=0.001):
    # Find a misclassified random noise.
    success = 0
    while not success:
        random_noise = torch.rand(X.shape)
        success = torch.argmax(model(norm((random_noise)))).item() != y

    # Binary search to minimize l2 distance to original image.
    alpha_low = 0.0
    alpha_up = 1.0
    while np.abs(alpha_low - alpha_up) > thresh:
        alpha_med = 0.5 * (alpha_low + alpha_up)
        tmp_x = (1 - alpha_med) * X + alpha_med * random_noise
        success = torch.argmax(model(norm((tmp_x)))).item() != y
        if success:
            alpha_up = alpha_med
        else:
            alpha_low = alpha_med

    initialization = (1 - alpha_up) * X + alpha_up * random_noise

    return initialization


def bin_search(model,  norm, X, currentX, y,  eta, grad, thresh=0.001):
    """TO COMPLETE: approach the boundary via a binary search."""
    pass


def montecarlo_grad(model,  norm, X, y, B, delta):
    """ TO COMPLETE: estimate the gradient"""
    grad /= torch.norm(grad, p=2)
    return grad


def hopskipjump(model,  norm, X, x0, y, n_steps=50, B0=100):
    """ Run Hop Skip Jump for L2 distance in an untargeted setting"""
    current_x = x0
    d = torch.tensor(int(np.prod(X.shape)))
    theta = 1.0 / (np.sqrt(d) * d)
    for k in trange(1, n_steps+1):
        dist = torch.norm(current_x-X, p=2)
        if k == 1:
            delta = 0.1
        else:
            delta = torch.sqrt(d) * theta * dist
        B = int(B0 * np.sqrt(k))
        grad = montecarlo_grad(model, norm, X, y, B, delta)
        eta = dist / np.sqrt(k)
        success = torch.argmax(model(norm(current_x + eta * grad))).item() != y
        while success:
            eta /= 2
            success = torch.argmax(
                model(norm(current_x + eta * grad))).item() != y
        current_x = bin_search(model, norm, X, current_x, y, eta, grad)
    return current_x

#### Train a CNN on MNIST

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True,
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor()
)

BATCH_SIZE = 32
# data loader
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1, 3*3*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)

    def forward(self, x):
        return x  # no normalization needed


# values are standard normalization for ImageNet images,
# from https://github.com/pytorch/examples/blob/master/imagenet/main.py
norm = Normalize(mean=[0], std=[1])
cnn = CNN()

In [ ]:
def train(model, train_loader):
    # ,lr=0.001, betas=(0.9,0.999))
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 1
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()
            predicted = torch.max(output.data, 1)[1]
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))


def evaluate(model, test_loader):
    correct = 0
    for test_imgs, test_labels in test_loader:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output, 1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}%".format(
        float(correct*100) / (len(test_loader)*BATCH_SIZE)))

In [ ]:
train(cnn, train_loader)

In [ ]:
evaluate(cnn, test_loader)

In [ ]:
# perform the black-box attack on the trained CNN
image3 = (train_data.data[train_data.targets == 3][0]).reshape(1, 1, 28, 28)
print("Predicted label before attack: ", cnn(
    torch.tensor(image3, dtype=torch.float)).argmax().detach().item())


x0 = initialize(cnn, norm, image3, 3)
print("Predicted label of adversarial init before HopSkipJump: ",
      cnn(x0).argmax().detach().item())
display_attack(image3, x0)
display_diff(image3, x0)

image_adv = hopskipjump(cnn, norm,  image3, x0, 3)
print("Predicted label of adversarial example after HopSkipJump: ",
      cnn(image_adv).argmax().detach().item())
display_attack(image3, image_adv)
display_diff(image3, image_adv)